In [1]:
import torch
import numpy as np

from transformers.models.wav2vec2.modeling_wav2vec2 import _compute_mask_indices, _sample_negative_indices
# from datasets import load_dataset
from lightning.pytorch import seed_everything
from transformers import Wav2Vec2Config
import seisbench.data as sbd

# from transformers import AutoFeatureExtractor, Wav2Vec2ForPreTraining
from transformers import Wav2Vec2ForPreTraining as RefWav2Vec2ForPreTraining
from seisLM.model.nano_wav2vec2 import Wav2Vec2ForPreTraining
from seisLM.configs.pretrain import pretrain_config



/home/liu0003/miniconda3/envs/seisbench/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
batch_size = 1
sequence_length = 5998
G = 2
V = 320

In [8]:
hidden_states = torch.randn(batch_size, sequence_length * G, V)
hidden_states = hidden_states.view(
  batch_size * sequence_length * G, -1
)

In [20]:
hidden_states.argmax(dim=-1, keepdim=True).shape

torch.Size([11996, 1])

In [27]:
import einops

codevector_idx = hidden_states.argmax(dim=-1)

# codevector_probs: [batch_size * sequence_length * G, V]

# hidden_states: [batch_size * sequence_length * G, V]

codevector_probs = hidden_states.new_zeros(hidden_states.shape).scatter_(
  -1, codevector_idx.view(-1, 1), 1.0
)

codevector_probs = einops.rearrange(
  codevector_probs, '(b l g) v -> (b l) g v',
  b=batch_size,
  l=sequence_length,
  g=G
)



codevector_probs = codevector_probs.view(batch_size * sequence_length, -1)
codevector_probs.shape

torch.Size([5998, 640])

In [33]:
codevector_dim = 240

codevectors = torch.rand((1, G * V,
  codevector_dim // G)
)

In [22]:
codevector_probs.unsqueeze(-1) * self.codevectors

torch.Size([11996, 320])

In [14]:
codevector_probs.shape

torch.Size([11996, 320])

In [5]:
data = sbd.STEAD()
waveforms = data.get_waveforms(1265656)
input_values = torch.Tensor(waveforms[0]).unsqueeze(0)
batch_size, raw_sequence_length = input_values.shape

2024-06-13 13:41:25,698 | seisbench | WARNING | Component order not specified, defaulting to 'ZNE'.


In [9]:
# config = pretrain_config.get_model_config()

# import ml_collections
# from typing import Any, List

# isinstance(config, List)

# isinstance(config, ml_collections.ConfigDict)

False

In [10]:

ref_model = RefWav2Vec2ForPreTraining(config)
nano_model = Wav2Vec2ForPreTraining(config)
nano_model.load_state_dict(ref_model.state_dict())

model_dict = {'ref_model': ref_model, 'nano_model': nano_model}
output_dict = {}
for model_name, model in model_dict.items():
  sequence_length = model._get_feat_extract_output_lengths(raw_sequence_length).item()

  seed_everything(0)
  mask_time_indices = _compute_mask_indices(
      shape=(batch_size, sequence_length), mask_prob=0.2, mask_length=2
  )
  sampled_negative_indices = _sample_negative_indices(
      features_shape=(batch_size, sequence_length),
      num_negatives=model.config.num_negatives,
      mask_time_indices=mask_time_indices,
  )
  mask_time_indices = torch.tensor(data=mask_time_indices, device=input_values.device, dtype=torch.long)
  sampled_negative_indices = torch.tensor(
      data=sampled_negative_indices, device=input_values.device, dtype=torch.long
  )

  with torch.no_grad():
    outputs = model(input_values, mask_time_indices=mask_time_indices,
                    sampled_negative_indices=sampled_negative_indices)
  output_dict[model_name] = outputs


Seed set to 0
Seed set to 0


NameError: name 'config' is not defined

In [12]:
for field in output_dict['ref_model']:
  print(field)
  value1 = getattr(output_dict['ref_model'], field)
  value2 = getattr(output_dict['nano_model'], field)
  assert np.allclose(value1.cpu().numpy(), value2.cpu().numpy())

loss
projected_states
projected_quantized_states
codevector_perplexity
contrastive_loss
diversity_loss


In [10]:
from torch import Tensor
from jaxtyping import Float32, jaxtyped
from typeguard import typechecked as typechecker


@jaxtyped(typechecker=typechecker)
def f(x: Float32[Tensor, "3 4"]) -> Float32[Tensor, "dim1 dim2"]:
    return x

In [12]:
import torch

f(torch.ones((1,2)))

TypeCheckError: Type-check error whilst checking the parameters of __main__.f.
The problem arose whilst typechecking parameter 'x'.
Actual value: tensor([[1., 1.]])
Expected type: <class 'Float32[Tensor, '3 4']'>.
----------------------
Called with parameters: {'x': tensor([[1., 1.]])}
Parameter annotations: (x: Float32[Tensor, '3 4']) -> Any.
